<a href="https://colab.research.google.com/github/Okitrader/freeCodeCampAlgoTrading/blob/main/FCC_Algo_Proj01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Packages Needed:
pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [1]:
!pip install pandas warnings pandas_ta numpy matplotlib statsmodels pandas-datareader datetime yfinance scikit-learn PyPortfolioOpt


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [2]:
!pip install pandas_ta requests tqdm

In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta
import warnings
warnings.filterwarnings('ignore')


## Create a list of top Cryptos Ranked by Coinmarket Cap.

In [4]:
# Import necessary libraries
import requests
import pandas as pd
from google.colab import userdata

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')

if api_key is None:
    print("API key not found. Please check that the secret name is correct and that the notebook has access to it.")
else:
    print("API key retrieved successfully.")

# Set the limit for the number of cryptocurrencies to retrieve
TOP_CRYPTO_LIMIT = 125  # Change this number to fetch a different number of top cryptocurrencies

# CoinMarketCap API URL and parameters
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start': '1',
    'limit': str(TOP_CRYPTO_LIMIT),  # Convert the limit to a string to use in the parameters
    'convert': 'USD'
}
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

# Make the API request
try:
    response = requests.get(url, headers=headers, params=parameters)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    data = response.json()

    # Parse the data to extract 'cmc_rank' and 'symbol'
    cryptos = [{'cmc_rank': entry['cmc_rank'], 'symbol': entry['symbol']} for entry in data['data']]

    # Create DataFrame and name it top_cryptos (renamed to reflect the variable limit)
    top_cryptos = pd.DataFrame(cryptos)

    # Verify that the ranks are within the top limit set
    if top_cryptos['cmc_rank'].max() <= TOP_CRYPTO_LIMIT and top_cryptos['cmc_rank'].min() >= 1:
        print(f"Successfully retrieved top {TOP_CRYPTO_LIMIT} cryptocurrencies by cmc_rank.")
    else:
        print(f"Some cryptocurrencies may not be in the top {TOP_CRYPTO_LIMIT} by cmc_rank.")

    # Display the DataFrame
    print(top_cryptos.head())
    print(top_cryptos.tail())
except requests.exceptions.HTTPError as errh:
    print("Http Error:", errh)
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
except requests.exceptions.RequestException as err:
    print("Oops: Something Else", err)

API key retrieved successfully.
Successfully retrieved top 125 cryptocurrencies by cmc_rank.
   cmc_rank symbol
0         1    BTC
1         2    ETH
2         3   USDT
3         4    BNB
4         5    XRP
     cmc_rank symbol
120       121    LRC
121       122    CVX
122       123  FLOKI
123       124    SFP
124       125   MASK


## Next we create a Dataframe for our OHLCV
we are not using adjusted close as in the video examples, hence our formulas will have to change with our version

In [5]:
from tqdm import tqdm
import pprint as pp
import datetime
pd.set_option('display.max_columns', None)
from google.colab import userdata # Ensure this contains your CoinMarketCap API key

# Retrieve API key from Colab user data
api_key = userdata.get('CMCKey')

# Function to fetch and process historical cryptocurrency data for each symbol
def fetch_historical_crypto_data(start_date=None, end_date=None, days_back=29):
    # Default to current date if end_date is not specified
    if end_date is None:
        end_date = datetime.datetime.now()

    # Default to 29 days back if start_date is not specified
    if start_date is None:
        start_date = end_date - datetime.timedelta(days=days_back)

    # Convert dates to string format if they are datetime objects
    if isinstance(start_date, datetime.datetime):
        start_date = start_date.strftime('%Y-%m-%d')
    if isinstance(end_date, datetime.datetime):
        end_date = end_date.strftime('%Y-%m-%d')

    base_url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    all_data = []

    # Using tqdm for progress display
    for symbol in tqdm(top_cryptos['symbol'], desc="Processing", unit="symbol"):
        url = f'{base_url}?symbol={symbol}'
        parameters = {
            'time_start': start_date,
            'time_end': end_date,
            'convert': 'USD'
        }

        try:
            response = requests.get(url, headers=headers, params=parameters)
            data = response.json()

            # Extract and flatten the quotes data
            quotes = data.get('data', {}).get(symbol, [])
            for quote in quotes:
                for q in quote.get('quotes', []):
                    usd_data = q['quote']['USD']
                    entry = {
                        'symbol': symbol,
                        'timestamp': usd_data['timestamp'],
                        'open': usd_data['open'],
                        'high': usd_data['high'],
                        'low': usd_data['low'],
                        'close': usd_data['close'],
                        'volume': usd_data['volume']
                    }
                    all_data.append(entry)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Convert all_data to a DataFrame
    top_OHLCV = pd.DataFrame(all_data)

    # Set the index to a MultiIndex of 'timestamp' and 'symbol'
    top_OHLCV.set_index(['timestamp', 'symbol'], inplace=True)

    # Ensure the column names are lowercase
    top_OHLCV.columns = top_OHLCV.columns.str.lower()

    # Round the volume column to 4 decimal places
    top_OHLCV['volume'] = top_OHLCV['volume'].round(6)

    # Return the final DataFrame
    return top_OHLCV

# Fetching data
top_OHLCV = fetch_historical_crypto_data()

# Print the data using pretty print
pp.pprint(top_OHLCV.head())
pp.pprint(top_OHLCV.tail())

Processing: 100%|██████████| 125/125 [00:34<00:00,  3.59symbol/s]


                                         open          high           low  \
timestamp                symbol                                             
2023-10-14T23:59:59.999Z BTC     26866.203245  26968.999218  26814.586586   
2023-10-15T23:59:59.999Z BTC     26858.011726  27289.170319  26817.894010   
2023-10-16T23:59:59.999Z BTC     27162.628229  29448.139037  27130.473478   
2023-10-17T23:59:59.999Z BTC     28522.098166  28618.752390  28110.186117   
2023-10-18T23:59:59.999Z BTC     28413.530808  28889.009589  28174.252551   

                                        close        volume  
timestamp                symbol                              
2023-10-14T23:59:59.999Z BTC     26861.706203  5.388117e+09  
2023-10-15T23:59:59.999Z BTC     27159.652919  7.098202e+09  
2023-10-16T23:59:59.999Z BTC     28519.466679  2.783388e+10  
2023-10-17T23:59:59.999Z BTC     28415.748140  1.487253e+10  
2023-10-18T23:59:59.999Z BTC     28328.341152  1.272413e+10  
                          

## Calculate features and technical indicators for each Crypto:
- Garman-Klass Volatility
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

The updated Garman-Klass Volatility formula for continuous markets like currencies, using Close instead of Adjusted Close, is given by:

Garman-Klass Volatility = ((ln(High) - ln(Low))^2 / 2) - (2ln(2) - 1)(ln(Close) - ln(Open))^2



In [6]:
# Garman-Klass Volatility
top_OHLCV['garman_klass_vol'] = ((np.log(top_OHLCV['high']) - np.log(top_OHLCV['low']))**2) / 2 - (2 * np.log(2) - 1) * ((np.log(top_OHLCV['close']) - np.log(top_OHLCV['open']))**2)

# RSI
top_OHLCV['rsi'] = top_OHLCV.groupby(level=1)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

top_OHLCV


,,open,high,low,close,volume,garman_klass_vol,rsi
timestamp,symbol,,,,,,,
2023-10-14T23:59:59.999Z,BTC,26866.203245,26968.999218,26814.586586,26861.706203,5.388117e+09,0.000016,NaN
2023-10-15T23:59:59.999Z,BTC,26858.011726,27289.170319,26817.894010,27159.652919,7.098202e+09,0.000104,NaN
2023-10-16T23:59:59.999Z,BTC,27162.628229,29448.139037,27130.473478,28519.466679,2.783388e+10,0.002442,NaN
2023-10-17T23:59:59.999Z,BTC,28522.098166,28618.752390,28110.186117,28415.748140,1.487253e+10,0.000155,NaN
2023-10-18T23:59:59.999Z,BTC,28413.530808,28889.009589,28174.252551,28328.341152,1.272413e+10,0.000310,NaN
...,...,...,...,...,...,...,...,...
2023-11-06T23:59:59.999Z,IMX,0.929718,1.146016,0.877298,0.910972,2.672363e+08,0.035538,85.526494
2023-11-07T23:59:59.999Z,IMX,0.911451,0.914076,0.805009,0.851145,8.999598e+07,0.006262,74.071262
2023-11-08T23:59:59.999Z,IMX,0.851138,0.863900,0.829301,0.856404,4.969280e+07,0.000821,74.388702


### Now calculate the Bollinger Bands

In [7]:
import numpy as np

def compute_bollinger_bands(df, length=20, num_std=2):
    """
    Calculate Bollinger Bands for a given DataFrame.

    Parameters:
    df (DataFrame): DataFrame with 'close' prices.
    length (int): The number of periods for the moving average.
    num_std (float): The number of standard deviations from the moving average.

    Returns:
    DataFrame: DataFrame with Bollinger Band columns added.
    """

    # Adjust the number of standard deviations based on the length
    if length > 20:
        num_std = 2.1  # Increasing for length > 20
    elif length < 20:
        num_std = 1.9  # Decreasing for length < 20

    # Calculate the moving average (middle band)
    df[f'bb_mid_{length}'] = df['close'].rolling(window=length, min_periods=1).mean()

    # Calculate the standard deviation
    rolling_std = df['close'].rolling(window=length, min_periods=1).std()

    # Calculate upper and lower bands
    df[f'bb_high_{length}'] = df[f'bb_mid_{length}'] + (rolling_std * num_std)
    df[f'bb_low_{length}'] = df[f'bb_mid_{length}'] - (rolling_std * num_std)

    return df

# Apply the function to your DataFrame
top_OHLCV = compute_bollinger_bands(top_OHLCV, length=20)  # Example usage with length=20
top_OHLCV

,,open,high,low,close,volume,garman_klass_vol,rsi,bb_mid_20,bb_high_20,bb_low_20
timestamp,symbol,,,,,,,,,,
2023-10-14T23:59:59.999Z,BTC,26866.203245,26968.999218,26814.586586,26861.706203,5.388117e+09,0.000016,NaN,26861.706203,NaN,NaN
2023-10-15T23:59:59.999Z,BTC,26858.011726,27289.170319,26817.894010,27159.652919,7.098202e+09,0.000104,NaN,27010.679561,27432.039848,26589.319274
2023-10-16T23:59:59.999Z,BTC,27162.628229,29448.139037,27130.473478,28519.466679,2.783388e+10,0.002442,NaN,27513.608600,29281.099384,25746.117816
2023-10-17T23:59:59.999Z,BTC,28522.098166,28618.752390,28110.186117,28415.748140,1.487253e+10,0.000155,NaN,27739.143485,29441.065398,26037.221572
2023-10-18T23:59:59.999Z,BTC,28413.530808,28889.009589,28174.252551,28328.341152,1.272413e+10,0.000310,NaN,27856.983018,29422.271094,26291.694943
...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06T23:59:59.999Z,IMX,0.929718,1.146016,0.877298,0.910972,2.672363e+08,0.035538,85.526494,0.665574,0.889818,0.441329
2023-11-07T23:59:59.999Z,IMX,0.911451,0.914076,0.805009,0.851145,8.999598e+07,0.006262,74.071262,0.682333,0.909577,0.455089
2023-11-08T23:59:59.999Z,IMX,0.851138,0.863900,0.829301,0.856404,4.969280e+07,0.000821,74.388702,0.699240,0.925401,0.473079


In [8]:
# Print columns of the original DataFrame to confirm 'close' exists
print(top_OHLCV.columns)

# If 'close' is present, reset the index
top_OHLCV_reset = top_OHLCV.reset_index()

# Print columns of the reset DataFrame to confirm 'close' still exists
print(top_OHLCV_reset.columns)


Index(['open', 'high', 'low', 'close', 'volume', 'garman_klass_vol', 'rsi',
       'bb_mid_20', 'bb_high_20', 'bb_low_20'],
      dtype='object')
Index(['timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume',
       'garman_klass_vol', 'rsi', 'bb_mid_20', 'bb_high_20', 'bb_low_20'],
      dtype='object')


In [9]:
import pandas as pd
import pandas_ta

# Reset the index to make 'symbol' a column
top_OHLCV_reset = top_OHLCV.reset_index()

# Define the compute_atr function
def compute_atr(crypto_data, length=14):
    crypto_data = crypto_data.sort_values(by='timestamp')
    atr = pandas_ta.atr(high=crypto_data['high'], low=crypto_data['low'], close=crypto_data['close'], length=length)
    normalized_atr = atr.sub(atr.mean()).div(atr.std())
    return normalized_atr

# Apply the compute_atr function for each 'symbol'
atr_values = top_OHLCV_reset.groupby('symbol').apply(lambda x: compute_atr(x, length=14))
# Drop level if it's a MultiIndex
if isinstance(atr_values.index, pd.MultiIndex):
    atr_values = atr_values.droplevel(0).rename('atr')

# Add the ATR values to the original DataFrame
top_OHLCV_reset['atr'] = atr_values

# Set the index back to 'timestamp' and 'symbol'
top_OHLCV_atr = top_OHLCV_reset.set_index(['timestamp', 'symbol'])

# Select only the desired columns after adding ATR
final_columns_atr = ['open', 'high', 'low', 'close', 'volume', 'garman_klass_vol', 'rsi',
                     'bb_mid_20', 'bb_high_20', 'bb_low_20', 'atr']
top_OHLCV = top_OHLCV_atr[final_columns_atr]

# Display the head of the updated DataFrame with ATR
print(top_OHLCV_atr.head())


                                         open          high           low  \
timestamp                symbol                                             
2023-10-14T23:59:59.999Z BTC     26866.203245  26968.999218  26814.586586   
2023-10-15T23:59:59.999Z BTC     26858.011726  27289.170319  26817.894010   
2023-10-16T23:59:59.999Z BTC     27162.628229  29448.139037  27130.473478   
2023-10-17T23:59:59.999Z BTC     28522.098166  28618.752390  28110.186117   
2023-10-18T23:59:59.999Z BTC     28413.530808  28889.009589  28174.252551   

                                        close        volume  garman_klass_vol  \
timestamp                symbol                                                 
2023-10-14T23:59:59.999Z BTC     26861.706203  5.388117e+09          0.000016   
2023-10-15T23:59:59.999Z BTC     27159.652919  7.098202e+09          0.000104   
2023-10-16T23:59:59.999Z BTC     28519.466679  2.783388e+10          0.002442   
2023-10-17T23:59:59.999Z BTC     28415.748140  1.487253

In [10]:
print(top_OHLCV.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'garman_klass_vol', 'rsi',
       'bb_mid_20', 'bb_high_20', 'bb_low_20', 'atr'],
      dtype='object')


## Calculating Dollar Volume
We calculate it by multiplying the 'close' price by the 'volume' of trades and then dividing the product by 100,000.

In [11]:
# Calculate the dollar volume
top_OHLCV['dollar_volume'] = (top_OHLCV['close'] * top_OHLCV['volume']) / 1e5 #

# Display the head of the updated DataFrame with the new 'dollar_volume' column
top_OHLCV.head()


,,open,high,low,close,volume,garman_klass_vol,rsi,bb_mid_20,bb_high_20,bb_low_20,atr,dollar_volume
timestamp,symbol,,,,,,,,,,,,
2023-10-14T23:59:59.999Z,BTC,26866.203245,26968.999218,26814.586586,26861.706203,5.388117e+09,0.000016,NaN,26861.706203,NaN,NaN,NaN,1.447340e+09
2023-10-15T23:59:59.999Z,BTC,26858.011726,27289.170319,26817.894010,27159.652919,7.098202e+09,0.000104,NaN,27010.679561,27432.039848,26589.319274,NaN,1.927847e+09
2023-10-16T23:59:59.999Z,BTC,27162.628229,29448.139037,27130.473478,28519.466679,2.783388e+10,0.002442,NaN,27513.608600,29281.099384,25746.117816,NaN,7.938073e+09
2023-10-17T23:59:59.999Z,BTC,28522.098166,28618.752390,28110.186117,28415.748140,1.487253e+10,0.000155,NaN,27739.143485,29441.065398,26037.221572,NaN,4.226140e+09
2023-10-18T23:59:59.999Z,BTC,28413.530808,28889.009589,28174.252551,28328.341152,1.272413e+10,0.000310,NaN,27856.983018,29422.271094,26291.694943,NaN,3.604535e+09


In [12]:
print(top_OHLCV.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'garman_klass_vol', 'rsi',
       'bb_mid_20', 'bb_high_20', 'bb_low_20', 'atr', 'dollar_volume'],
      dtype='object')


## Calculating Moving Average Convergence Divergence (MACD) in Python
Calculate the Moving Average Convergence Divergence (MACD), adapted from a method found on AlphaRithms.
 Our implementation involves defining a function, compute_macd, which calculates the MACD by determining the difference between the 12-day and 26-day Exponential Moving Averages (EMAs) of closing prices. It further computes the Signal line and the MACD Histogram, which is normalized for suitability in machine learning models. We apply this function to each symbol in our multi-indexed top_OHLCV DataFrame, which is first reset and reindexed by 'timestamp' and 'symbol'. The process culminates in an aggregated DataFrame, macd_df, containing the MACD values alongside other financial data, showcasing the unique financial trends of each symbol in the dataset.

In [13]:
# adapted this blog for my function https://www.alpharithms.com/calculate-macd-python-272222/
import pandas as pd
import pandas_ta

# Define the compute_macd function
def compute_macd(close):
    macd = close.ewm(span=12, adjust=False).mean() - close.ewm(span=26, adjust=False).mean()
    macd_signal = macd.ewm(span=9, adjust=False).mean()
    macd_histogram = macd - macd_signal
    normalized_macd = macd_histogram.sub(macd_histogram.mean()).div(macd_histogram.std())
    return normalized_macd

# Reset the current index of the DataFrame
top_OHLCV_reset = top_OHLCV.reset_index()

# Set a new multi-index using 'timestamp' and 'symbol'
top_OHLCV_reset.set_index(['timestamp', 'symbol'], inplace=True)

# Initialize a DataFrame to store MACD values
macd_df = pd.DataFrame()

# Iterate over each group, apply compute_macd function, and aggregate results
for symbol, group in top_OHLCV_reset.groupby(level='symbol'):
    group['macd'] = compute_macd(group['close'])
    macd_df = macd_df.append(group)

# Now macd_df contains the MACD values along with other data
top_OHLCV_final = macd_df

# Display the head of the updated DataFrame
print(top_OHLCV_final[['close', 'macd']].head())
# THIS IS SEPERATE DATAFRAME. STILL TRYING TO WORK ON THE MULTIINDEX ISSUE

                                     close      macd
timestamp                symbol                     
2023-10-14T23:59:59.999Z AAVE    64.339649 -1.279870
2023-10-15T23:59:59.999Z AAVE    63.985461 -1.309893
2023-10-16T23:59:59.999Z AAVE    64.887691 -1.250591
2023-10-17T23:59:59.999Z AAVE    62.667519 -1.403148
2023-10-18T23:59:59.999Z AAVE    62.098257 -1.540277


## Aggregate Crypto Liquidity for the Top assests
In-depth framework for aggregating, visualizing, and analyzing monthly cryptocurrency data. It encompasses methods for computing rolling averages, identifying the most liquid cryptocurrencies, and equipping advanced financial indicators for comprehensive crypto market analysis.


In [16]:
# First, ensure the DataFrame is indexed correctly
top_OHLCV.reset_index(inplace=True)

# Ensure 'timestamp' is a datetime column
top_OHLCV['timestamp'] = pd.to_datetime(top_OHLCV['timestamp'])

# Set the index to 'timestamp' and 'symbol'
top_OHLCV.set_index(['timestamp', 'symbol'], inplace=True)

# Aggregate the data to ensure unique index
top_OHLCV_agg = top_OHLCV.groupby(['timestamp', 'symbol']).mean()

# Now, resample and calculate the daily mean of 'dollar_volume'
# Ensure the index is a DatetimeIndex for resampling
top_OHLCV_agg = top_OHLCV_agg.unstack('symbol')
daily_dollar_volume = top_OHLCV_agg['dollar_volume'].resample('D').mean().stack('symbol').to_frame('dollar_volume')

# Resample and get the last values of the other columns for each day
daily_last_values = top_OHLCV_agg.drop(columns='dollar_volume').resample('D').last().stack('symbol')

# Combine the two DataFrames
data_daily = pd.concat([daily_dollar_volume, daily_last_values], axis=1).dropna()

# Display the resulting DataFrame
print(data_daily.head())


                                  dollar_volume       atr  bb_high_20  \
timestamp                 symbol                                        
2023-10-18 00:00:00+00:00 ARB        242.126808 -0.140550    2.601742   
                          DOGE        22.782585  0.073113    0.234415   
                          TON        246.651037  1.308465    2.707457   
2023-10-19 00:00:00+00:00 ARB        250.864822 -1.726310    2.627132   
                          DOGE        23.670665 -1.706151    0.236016   

                                  bb_low_20  bb_mid_20     close  \
timestamp                 symbol                                   
2023-10-18 00:00:00+00:00 ARB      0.287391   1.444567  0.525296   
                          DOGE     0.020744   0.127580  0.039106   
                          TON      0.356002   1.531730  1.829399   
2023-10-19 00:00:00+00:00 ARB      0.175435   1.401284  0.530415   
                          DOGE     0.008886   0.122451  0.039215   

           